In [206]:
import spotipy
import pandas as pd
import numpy as np
import spotipy.util as util
import os
from json.decoder import JSONDecodeError
from urllib.error import HTTPError
from spotipy import SpotifyException

In [208]:
def getToken():
    client_id = '23d505f16e00474998680af9c6d86270' # Your client id
    client_secret = '2586a692d95348d08b5d5474e4bbf7b6' # Your secret
    redirect_uri = 'https://k-next.com/callback' # Your redirect uri
    username = '22gbdmdtseyf5q7vdfo7l7eha'
    # Get OAuth token
    try:
        token = util.prompt_for_user_token(username=username, scope='user-follow-read', client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri)
    except(AttributeError, JSONDecodeError):
        os.remove(f".cache-{username}")
        token = util.prompt_for_user_token(username=username, scope='user-follow-read', client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri)

    # Initialize Spotipy with the token obtained above
    sp = spotipy.Spotify(auth=token)
    print(token)
    return sp

In [420]:
sp = getToken()

BQCGxAnfvGRvplRtewMOxyaOihlWQ8WRc0ksDNvloyHil7Jp5zCeTP9x4aLJjgCSVTPhDTMMygJcOEctJRoW2UEHOd3M6uKkGyWEicb5NhK3rWebKL8h-UxF5BEhKwhbJWOSiaWGe-kwGMJuBxY6-f4_UOsR9BG8hUdT9iYhggcEEDW2Vy4BScAukQ


In [171]:
artistList = []
artists = sp.current_user_followed_artists(limit=50, after=None)
artists = artists.get('artists')
artistList.extend(artists.get('items'))
while artists.get('next'):
    artists = sp.next(artists)
    artists = artists.get('artists')
    artistList.extend(artists.get('items'))

In [213]:
albumList = []
for artist in artistList:
    albums = sp.artist_albums(artist.get('id'), album_type=None, country=None, limit=50, offset=0)
    albumList.extend(albums.get('items'))
    while albums.get('next'):
        albums = sp.next(albums)
        albumList.extend(albums.get('items'))        

In [175]:
songList = []
for album in albumList:
    songs = sp.album_tracks(album.get('id'), limit=50, offset=0)
    songList.extend(songs.get('items'))
    while songs.get('next'):
        songs = sp.next(songs)
        songList.extend(songs.get('items'))

In [324]:
# DataFrame to store audio features
df = pd.DataFrame(columns=['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])

In [325]:
for song in songList:
    trackID = song.get('id')
    try:
        features = sp.audio_features(tracks=[trackID])
    except(HTTPError, SpotifyException):
        sp = getToken()
        features = sp.audio_features(tracks=[trackID])
    df = df.append(features)
df.head(5)

BQDsSDVJ2VsoFTxcFsJDV-DPVpQEBJfDgov-paxPYhu0_NM_GnrOcg2LGBqVkh4zzt6_OH2DFlCdp6I3BAAfQ-ATR2lfgY6iIp7C2xnM66k_5c4Lj3wCE_8t4oUkKNhwv1QU85hL6PAjv_CER8t36kNIkbhcNKbghTtVLJAtyNRaPRl35CGDSMd0Rg


/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:107: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)
/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


BQBLLZa1YJ2f7mBL0UAsMfZ3Pr58ug5Smurht1bKTr6lCKPO_aMNcRI5ac9i2TmW85IGfVIyPoHUns2FXVINcGJIgbWi5FwNo7xO3KbYMrBXLA8qGRLKXiSiQCVVmGjJjv1SucmpvHHOzFCGidbzzYivWPTK63----bFpDV7HouSH_xNLNCzdtj7tQ
BQDD6rcWnOaU2N6El35IOob0YeKmM5_FTV1l_fnm2asWL2Xm1a4CpihP3DtpcIppcrZR-CQHaAdGm9WQ38wW28JXy5N9rr1BRZ39FYVrPpR-wyeJpVvbrDS-MHQZkantBPR2C9HKr9DzxKauVDW0sDdp1myoFtNq1jB5uaMuIIM-I3EiGYWivDyFVA


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,0
0,0.621,0.903,11,-2.602,1,0.2780,0.5270,0.00026,0.1050,0.829,164.995,audio_features,4cARnmpAHfUNj23dfenBUu,spotify:track:4cARnmpAHfUNj23dfenBUu,https://api.spotify.com/v1/tracks/4cARnmpAHfUN...,https://api.spotify.com/v1/audio-analysis/4cAR...,172291,4,NaN
0,0.684,0.855,11,-8.147,1,0.0899,0.0284,0.79500,0.0541,0.836,165.028,audio_features,7pWUAJfehiy77D43d8tX4R,spotify:track:7pWUAJfehiy77D43d8tX4R,https://api.spotify.com/v1/tracks/7pWUAJfehiy7...,https://api.spotify.com/v1/audio-analysis/7pWU...,172291,4,NaN
0,0.637,0.971,7,-5.458,1,0.1010,0.0316,0.00191,0.2100,0.695,144.053,audio_features,2NyyEWzSXIFkCmm4gFPmrr,spotify:track:2NyyEWzSXIFkCmm4gFPmrr,https://api.spotify.com/v1/tracks/2NyyEWzSXIFk...,https://api.spotify.com/v1/audio-analysis/2Nyy...,204309,4,NaN
0,0.761,0.938,6,-6.093,0,0.0884,0.1760,0.00000,0.0645,0.966,160.123,audio_features,5qKAPmk4y4bMqMb4nIK9uX,spotify:track:5qKAPmk4y4bMqMb4nIK9uX,https://api.spotify.com/v1/tracks/5qKAPmk4y4bM...,https://api.spotify.com/v1/audio-analysis/5qKA...,228484,4,NaN
0,0.638,0.775,11,-3.373,1,0.0491,0.1470,0.00000,0.7610,0.586,93.959,audio_features,04VuGhQMjyrVVYjtLzQYCV,spotify:track:04VuGhQMjyrVVYjtLzQYCV,https://api.spotify.com/v1/tracks/04VuGhQMjyrV...,https://api.spotify.com/v1/audio-analysis/04Vu...,262034,4,NaN


In [388]:
# trainData = pd.read_csv('/Users/akounoroushi/audio_features.csv')
# trainData.columns = ['','danceability', 'energy', 'key', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'id', '0']
trainData = df.drop(columns=['type', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'mode', 'time_signature'])
trainData.head(5)

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,0
0,0.621,0.903,11,-2.602,0.2780,0.5270,0.00026,0.1050,0.829,164.995,4cARnmpAHfUNj23dfenBUu,NaN
0,0.684,0.855,11,-8.147,0.0899,0.0284,0.79500,0.0541,0.836,165.028,7pWUAJfehiy77D43d8tX4R,NaN
0,0.637,0.971,7,-5.458,0.1010,0.0316,0.00191,0.2100,0.695,144.053,2NyyEWzSXIFkCmm4gFPmrr,NaN
0,0.761,0.938,6,-6.093,0.0884,0.1760,0.00000,0.0645,0.966,160.123,5qKAPmk4y4bMqMb4nIK9uX,NaN
0,0.638,0.775,11,-3.373,0.0491,0.1470,0.00000,0.7610,0.586,93.959,04VuGhQMjyrVVYjtLzQYCV,NaN


In [193]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [389]:
#trainData['loudness'] = pd.to_numeric(trainData['loudness'], errors='coerce')
#newCol = np.reshape(trainData['loudness'].tolist(), (-1, 1))
#newCol = scaler.fit_transform(newCol)
#newCol = newCol.reshape(-1)
#trainData['std_loudness'] = newCol
#trainData.head(5)
standardized = []
for index, row in trainData.iterrows():
    standardized.append((row['loudness'] - trainData['loudness'].mean()) / trainData['loudness'].std())
trainData['std_loudness'] = standardized

In [390]:
standardized = []
for index, row in trainData.iterrows():
    standardized.append((row['danceability'] - trainData['danceability'].mean()) / trainData['danceability'].std())
trainData['std_danceability'] = standardized

In [391]:
standardized = []
for index, row in trainData.iterrows():
    standardized.append((row['energy'] - trainData['energy'].mean()) / trainData['energy'].std())
trainData['std_energy'] = standardized

In [392]:
standardized = []
for index, row in trainData.iterrows():
    standardized.append((row['key'] - trainData['key'].mean()) / trainData['key'].std())
trainData['std_key'] = standardized

In [393]:
standardized = []
for index, row in trainData.iterrows():
    standardized.append((row['speechiness'] - trainData['speechiness'].mean()) / trainData['speechiness'].std())
trainData['std_speechiness'] = standardized

In [394]:
standardized = []
for index, row in trainData.iterrows():
    standardized.append((row['acousticness'] - trainData['acousticness'].mean()) / trainData['acousticness'].std())
trainData['std_acousticness'] = standardized

In [395]:
standardized = []
for index, row in trainData.iterrows():
    standardized.append((row['instrumentalness'] - trainData['instrumentalness'].mean()) / trainData['instrumentalness'].std())
trainData['std_instrumentalness'] = standardized

In [396]:
standardized = []
for index, row in trainData.iterrows():
    standardized.append((row['liveness'] - trainData['liveness'].mean()) / trainData['liveness'].std())
trainData['std_liveness'] = standardized

In [397]:
standardized = []
for index, row in trainData.iterrows():
    standardized.append((row['valence'] - trainData['valence'].mean()) / trainData['valence'].std())
trainData['std_valence'] = standardized

In [398]:
standardized = []
for index, row in trainData.iterrows():
    standardized.append((row['tempo'] - trainData['tempo'].mean()) / trainData['tempo'].std())
trainData['std_tempo'] = standardized
trainData.head(5)

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,...,std_loudness,std_danceability,std_energy,std_key,std_speechiness,std_acousticness,std_instrumentalness,std_liveness,std_valence,std_tempo
0,0.621,0.903,11,-2.602,0.2780,0.5270,0.00026,0.1050,0.829,164.995,...,1.109658,0.040933,1.003628,1.597711,1.905348,0.703676,-0.565008,-0.554933,1.284456,1.640197
0,0.684,0.855,11,-8.147,0.0899,0.0284,0.79500,0.0541,0.836,165.028,...,-0.326066,0.438564,0.802707,1.597711,-0.007698,-0.883162,1.760905,-0.816967,1.311356,1.641406
0,0.637,0.971,7,-5.458,0.1010,0.0316,0.00191,0.2100,0.695,144.053,...,0.370176,0.141918,1.288267,0.487120,0.105193,-0.872977,-0.560179,-0.014391,0.769501,0.873173
0,0.761,0.938,6,-6.093,0.0884,0.1760,0.00000,0.0645,0.966,160.123,...,0.205760,0.924557,1.150134,0.209472,-0.022954,-0.413412,-0.565769,-0.763428,1.810939,1.461755
0,0.638,0.775,11,-3.373,0.0491,0.1470,0.00000,0.7610,0.586,93.959,...,0.910029,0.148230,0.467838,1.597711,-0.422649,-0.505707,-0.565769,2.822170,0.350621,-0.961576


In [399]:
trainData = trainData.fillna(0)
trainData.head(5)

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,...,std_loudness,std_danceability,std_energy,std_key,std_speechiness,std_acousticness,std_instrumentalness,std_liveness,std_valence,std_tempo
0,0.621,0.903,11,-2.602,0.2780,0.5270,0.00026,0.1050,0.829,164.995,...,1.109658,0.040933,1.003628,1.597711,1.905348,0.703676,-0.565008,-0.554933,1.284456,1.640197
0,0.684,0.855,11,-8.147,0.0899,0.0284,0.79500,0.0541,0.836,165.028,...,-0.326066,0.438564,0.802707,1.597711,-0.007698,-0.883162,1.760905,-0.816967,1.311356,1.641406
0,0.637,0.971,7,-5.458,0.1010,0.0316,0.00191,0.2100,0.695,144.053,...,0.370176,0.141918,1.288267,0.487120,0.105193,-0.872977,-0.560179,-0.014391,0.769501,0.873173
0,0.761,0.938,6,-6.093,0.0884,0.1760,0.00000,0.0645,0.966,160.123,...,0.205760,0.924557,1.150134,0.209472,-0.022954,-0.413412,-0.565769,-0.763428,1.810939,1.461755
0,0.638,0.775,11,-3.373,0.0491,0.1470,0.00000,0.7610,0.586,93.959,...,0.910029,0.148230,0.467838,1.597711,-0.422649,-0.505707,-0.565769,2.822170,0.350621,-0.961576


In [422]:
trackIDs = ['20yOiJHRQGirFYehRSIb6i', '0CPWIY1etGeJLg12CAuP4n', '6jLsYKGOBRBRvdowIzuyUf', '3L5VjGkh1U9yRHsvshoqGY', '0ErzcmZ2gIwX7X0xSMQPix', '2JTjDYA0vfx8sp5XMTbeqU', '5RGf8qn1TBWVplLyyKUsuV', '7uW2sLxLjNgQJHa71LGVGr', '5s8hBT38j6rylFfX95nbVo', '6fzJaYb6xprozFzFdqfI9s', '0yiW0iUlPQg5htp6GyR7v7']
X_train = pd.DataFrame(columns=['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])
for trackID in trackIDs:
    features = sp.audio_features(tracks=[trackID])
    X_train = X_train.append(features)
X_train

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.729,0.928,7,-3.028,1,0.0540,0.05400,0,0.3600,0.618,115.998,audio_features,20yOiJHRQGirFYehRSIb6i,spotify:track:20yOiJHRQGirFYehRSIb6i,https://api.spotify.com/v1/tracks/20yOiJHRQGir...,https://api.spotify.com/v1/audio-analysis/20yO...,224931,4
0,0.786,0.783,5,-5.598,1,0.0451,0.15800,0,0.1090,0.449,106.987,audio_features,0CPWIY1etGeJLg12CAuP4n,spotify:track:0CPWIY1etGeJLg12CAuP4n,https://api.spotify.com/v1/tracks/0CPWIY1etGeJ...,https://api.spotify.com/v1/audio-analysis/0CPW...,203057,4
0,0.400,0.724,8,-6.064,1,0.2530,0.37500,0,0.0927,0.430,96.782,audio_features,6jLsYKGOBRBRvdowIzuyUf,spotify:track:6jLsYKGOBRBRvdowIzuyUf,https://api.spotify.com/v1/tracks/6jLsYKGOBRBR...,https://api.spotify.com/v1/audio-analysis/6jLs...,216786,4
0,0.638,0.905,0,-2.208,1,0.0454,0.09690,0,0.0714,0.471,139.975,audio_features,3L5VjGkh1U9yRHsvshoqGY,spotify:track:3L5VjGkh1U9yRHsvshoqGY,https://api.spotify.com/v1/tracks/3L5VjGkh1U9y...,https://api.spotify.com/v1/audio-analysis/3L5V...,205853,4
0,0.787,0.543,10,-6.700,0,0.0370,0.02330,0,0.0861,0.332,99.026,audio_features,0ErzcmZ2gIwX7X0xSMQPix,spotify:track:0ErzcmZ2gIwX7X0xSMQPix,https://api.spotify.com/v1/tracks/0ErzcmZ2gIwX...,https://api.spotify.com/v1/audio-analysis/0Erz...,210641,4
0,0.621,0.873,11,-2.888,0,0.1030,0.01940,0,0.4880,0.401,169.910,audio_features,2JTjDYA0vfx8sp5XMTbeqU,spotify:track:2JTjDYA0vfx8sp5XMTbeqU,https://api.spotify.com/v1/tracks/2JTjDYA0vfx8...,https://api.spotify.com/v1/audio-analysis/2JTj...,210478,4
0,0.583,0.885,0,-3.571,0,0.1060,0.02440,0,0.3570,0.601,92.959,audio_features,5RGf8qn1TBWVplLyyKUsuV,spotify:track:5RGf8qn1TBWVplLyyKUsuV,https://api.spotify.com/v1/tracks/5RGf8qn1TBWV...,https://api.spotify.com/v1/audio-analysis/5RGf...,217052,4
0,0.750,0.799,0,-2.788,0,0.0697,0.07060,0,0.3770,0.462,110.991,audio_features,7uW2sLxLjNgQJHa71LGVGr,spotify:track:7uW2sLxLjNgQJHa71LGVGr,https://api.spotify.com/v1/tracks/7uW2sLxLjNgQ...,https://api.spotify.com/v1/audio-analysis/7uW2...,206848,4
0,0.683,0.907,10,-3.375,0,0.0956,0.01390,0,0.7010,0.626,123.983,audio_features,5s8hBT38j6rylFfX95nbVo,spotify:track:5s8hBT38j6rylFfX95nbVo,https://api.spotify.com/v1/tracks/5s8hBT38j6ry...,https://api.spotify.com/v1/audio-analysis/5s8h...,210711,4
0,0.549,0.977,1,-1.443,0,0.1150,0.00735,0,0.0399,0.582,127.006,audio_features,6fzJaYb6xprozFzFdqfI9s,spotify:track:6fzJaYb6xprozFzFdqfI9s,https://api.spotify.com/v1/tracks/6fzJaYb6xpro...,https://api.spotify.com/v1/audio-analysis/6fzJ...,202520,4


In [423]:
X_train = X_train.drop(columns=['type', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'mode', 'time_signature'])
X_train.head(5)

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id
0,0.729,0.928,7,-3.028,0.0540,0.0540,0,0.3600,0.618,115.998,20yOiJHRQGirFYehRSIb6i
0,0.786,0.783,5,-5.598,0.0451,0.1580,0,0.1090,0.449,106.987,0CPWIY1etGeJLg12CAuP4n
0,0.400,0.724,8,-6.064,0.2530,0.3750,0,0.0927,0.430,96.782,6jLsYKGOBRBRvdowIzuyUf
0,0.638,0.905,0,-2.208,0.0454,0.0969,0,0.0714,0.471,139.975,3L5VjGkh1U9yRHsvshoqGY
0,0.787,0.543,10,-6.700,0.0370,0.0233,0,0.0861,0.332,99.026,0ErzcmZ2gIwX7X0xSMQPix


In [424]:
standardized = []
for index, row in X_train.iterrows():
    standardized.append((row['loudness'] - trainData['loudness'].mean()) / trainData['loudness'].std())
X_train['std_loudness'] = standardized

In [425]:
standardized = []
for index, row in X_train.iterrows():
    standardized.append((row['danceability'] - trainData['danceability'].mean()) / trainData['danceability'].std())
X_train['std_danceability'] = standardized

In [426]:
standardized = []
for index, row in X_train.iterrows():
    standardized.append((row['energy'] - trainData['energy'].mean()) / trainData['energy'].std())
X_train['std_energy'] = standardized

In [427]:
standardized = []
for index, row in X_train.iterrows():
    standardized.append((row['key'] - trainData['key'].mean()) / trainData['key'].std())
X_train['std_key'] = standardized

In [428]:
standardized = []
for index, row in X_train.iterrows():
    standardized.append((row['speechiness'] - trainData['speechiness'].mean()) / trainData['speechiness'].std())
X_train['std_speechiness'] = standardized

In [429]:
standardized = []
for index, row in X_train.iterrows():
    standardized.append((row['acousticness'] - trainData['acousticness'].mean()) / trainData['acousticness'].std())
X_train['std_acousticness'] = standardized

In [430]:
standardized = []
for index, row in X_train.iterrows():
    standardized.append((row['instrumentalness'] - trainData['instrumentalness'].mean()) / trainData['instrumentalness'].std())
X_train['std_instrumentalness'] = standardized

In [431]:
standardized = []
for index, row in X_train.iterrows():
    standardized.append((row['liveness'] - trainData['liveness'].mean()) / trainData['liveness'].std())
X_train['std_liveness'] = standardized

In [432]:
standardized = []
for index, row in X_train.iterrows():
    standardized.append((row['valence'] - trainData['valence'].mean()) / trainData['valence'].std())
X_train['std_valence'] = standardized

In [433]:
standardized = []
for index, row in X_train.iterrows():
    standardized.append((row['tempo'] - trainData['tempo'].mean()) / trainData['tempo'].std())
X_train['std_tempo'] = standardized

In [434]:
from sklearn.svm import OneClassSVM
svm = OneClassSVM()
svm = svm.fit(X=X_train[['std_loudness', 'std_danceability', 'std_energy', 'std_key', 'std_speechiness', 'std_acousticness', 'std_instrumentalness', 'std_liveness', 'std_valence', 'std_tempo']])
score = svm.decision_function(X=trainData[['std_loudness', 'std_danceability', 'std_energy', 'std_key', 'std_speechiness', 'std_acousticness', 'std_instrumentalness', 'std_liveness', 'std_valence', 'std_tempo']])
score = score.reshape(-1)
score = score.tolist()
trainData['score'] = score
trainData = trainData.sort_values(by=['score'], ascending=False)
trainData.head(5)

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,...,std_danceability,std_energy,std_key,std_speechiness,std_acousticness,std_instrumentalness,std_liveness,std_valence,std_tempo,score
0,0.651,0.756,8,-4.388,0.0762,0.0298,0.0,0.396,0.473,121.960,...,0.230281,0.388306,0.764768,-0.147032,-0.878706,-0.565769,0.943142,-0.083632,0.063992,0.725162
0,0.651,0.756,8,-4.388,0.0762,0.0298,0.0,0.396,0.473,121.960,...,0.230281,0.388306,0.764768,-0.147032,-0.878706,-0.565769,0.943142,-0.083632,0.063992,0.725162
0,0.594,0.931,5,-3.816,0.0948,0.1090,0.0,0.228,0.452,128.028,...,-0.129481,1.120833,-0.068176,0.042136,-0.626645,-0.565769,0.078274,-0.164334,0.286239,0.708595
0,0.668,0.830,8,-3.916,0.0537,0.0528,0.0,0.271,0.543,119.905,...,0.337578,0.698060,0.764768,-0.375866,-0.805507,-0.565769,0.299639,0.185374,-0.011275,0.697264
0,0.626,0.863,5,-5.201,0.1070,0.1230,0.0,0.196,0.529,130.121,...,0.072491,0.836194,-0.068176,0.166215,-0.582089,-0.565769,-0.086463,0.131573,0.362898,0.693182


In [435]:
print('Recommended songs:')
for index, row in trainData.iloc[0:15].iterrows():
    songID = row['id']
    song = sp.track(songID)
    artists = ''
    for artist in song.get('artists'):
        artists += artist.get('name') + ', '
    print('"' + song.get('name') + '" sung by ' + artists + ". Listen on Spotify: " + song.get('external_urls').get('spotify'))

Recommended songs:
"Best Of Me" sung by BTS, . Listen on Spotify: https://open.spotify.com/track/3KGEbCfnzAzFVimY9p62MN
"Best Of Me" sung by BTS, . Listen on Spotify: https://open.spotify.com/track/2tdq0CeAGSfhI2l4Gwkvgx
"Carry" sung by SNUPER, . Listen on Spotify: https://open.spotify.com/track/6FDT7SVrjZu6yWvR5FYrfc
"Good Evening - Japanese Ver." sung by SHINee, . Listen on Spotify: https://open.spotify.com/track/3qSQKTaFjSMrdQGzwaAgWz
"In my bed (feat.Brother Su)" sung by Mad Clown, . Listen on Spotify: https://open.spotify.com/track/4JP6o0htM49u1Dd5CKia8q
"Forever" sung by BewhY, . Listen on Spotify: https://open.spotify.com/track/4wFAvJ6hmcjEJPUw3ap2CM
"Forever" sung by BewhY, GRAY, . Listen on Spotify: https://open.spotify.com/track/2UbelwIXVqlm9lBkXygpsg
"Forever" sung by BewhY, . Listen on Spotify: https://open.spotify.com/track/4wFAvJ6hmcjEJPUw3ap2CM
"Masquerade - ArmySlick's Bavtronic Mix" sung by 2PM, . Listen on Spotify: https://open.spotify.com/track/2K7XQBIsiakTcWmjGr4QDU